# Data Wrangling, Cleaning for Seattle Public Library Checkouts
### continued from part 1: Library Collection
Dataset: The library checkouts for the entire Seattle Public Library system in 2016

Source: https://www.kaggle.com/seattle-public-library/seattle-library-checkout-records

In [1]:
import numpy as np
import pandas as pd

# style parameters
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Load Data
Load the cleaned library collection file from part 1, the checkout records and a dewey decimal system code dictionary

In [2]:
df = pd.read_csv('../../data/raw/Checkouts_By_Title_Data_Lens_2016.csv')
df_inv = pd.read_csv('../../data/processed/collection.csv')
df_dewey = pd.read_csv('../../data/raw/dewey.csv')

In [3]:
print(df.head())

,BibNumber,ItemBarcode,ItemType,Collection,CallNumber,CheckoutDateTime
0,3178331,10087139340,acdvd,nadvd,DVD WALLAND Season 4,07/30/2016 11:24:00 AM
1,2619159,10070409924,acbk,nycomic,YA 741.5973 B3903H02 2009,12/23/2016 01:04:00 PM
2,2406194,10086221842,acdvd,nadvd,DVD LAST KI,01/26/2016 01:53:00 PM
3,2655929,10074162446,acbk,canf,428.2 W863G 2010,06/03/2016 04:56:00 PM
4,3154003,10088095665,acdvd,nadvd,DVD STEVE J,11/19/2016 04:32:00 PM


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6404830 entries, 0 to 6404829
Data columns (total 6 columns):
BibNumber           int64
ItemBarcode         int64
ItemType            object
Collection          object
CallNumber          object
CheckoutDateTime    object
dtypes: int64(2), object(4)
memory usage: 293.2+ MB
None


### There are 6,404,830 checkouts in 2016 across the whole library system

In [5]:
df.isnull().sum()

BibNumber             0
ItemBarcode           0
ItemType              0
Collection            0
CallNumber          349
CheckoutDateTime      0
dtype: int64

### Removing Nulls

The checkout set is mostly clean except for 349 missing Call Numbers. Looking closely, the item borrowed the second most times with a missing call number is a modem for loan and the top one doesn't exist in the catalogue anymore so it will not appear in the final dataset anyway. We can go ahead and drop all the missing entries since it won't move the needle.

In [8]:
df[df['CallNumber'].isnull()==True]['BibNumber'].value_counts()

774693     255
3137313     58
2795292      9
2001737      8
3137570      6
2069856      4
406307       4
2444123      3
3202718      1
2852359      1
Name: BibNumber, dtype: int64

In [19]:
df_inv[df_inv['BibNum'].isin([3137313,774693])]

,BibNum,Title,Author,Publisher,ItemLocation,ItemCount,Age,Description,Format Group,Format Subgroup,Category Group,Child,Teen
617577,3137313,Connecting for Digital Literacy : modem / Veri...,NaN,"Verizon Wireless,",drp1,20,2,NA-Nonfiction,Print,Book,Nonfiction,0,0


In [20]:
df.dropna(inplace=True)
print('CallNumber nulls: ',df['CallNumber'].isnull().sum())

### Remove columns
We can get rid of the ItemBarcode, ItemType, and Collection columns because the inventory dataframe we will join contains the information

In [24]:
df.drop(['ItemBarcode','ItemType','Collection'],axis=1,inplace=True)
df.head(1)

,BibNumber,CallNumber,CheckoutDateTime
0,3178331,DVD WALLAND Season 4,07/30/2016 11:24:00 AM


### Change CheckoutDateTime to Datetime datatype

In [28]:
df['CheckoutDateTime'] = pd.to_datetime(df['CheckoutDateTime'], format= '%m/%d/%Y %I:%M:%S %p')

### Create month and day columns from CheckoutDateTime

In [29]:
df['CheckoutMonth'] = df['CheckoutDateTime'].dt.month
df['CheckoutDay'] = df['CheckoutDateTime'].dt.dayofweek
# df['CheckoutYear'] = df['CheckoutDateTime'].dt.year
print(df.columns)

Index(['BibNumber', 'CallNumber', 'CheckoutDateTime', 'CheckoutMonth',
       'CheckoutDay'],
      dtype='object')


### Create column for nonfiction dewey decimal category
The eventual deweySubject column will help with analyzing nonfiction titles.

In [31]:
# find 3 digit category from call number
df['dewey'] = df['CallNumber'].str.extract(r'(\d{3}\.)')
# remove period from number
df['dewey'] = df['dewey'].str.replace('.','')

In [33]:
df.head(1)

,BibNumber,CallNumber,CheckoutDateTime,CheckoutMonth,CheckoutDay,dewey
0,3178331,DVD WALLAND Season 4,2016-07-30 11:24:00,7,5,NaN


#### Change dewey column datatype
We change the dtype of dewey to int so that it can match in the join including numbers that look like 001. Changing dtype to integer yields an error so to work around we change all the null dewey numbers to -1 and try again. The dewey column will be dropped anyway after the subject has been obtained.

In [35]:
df['dewey'] = df['dewey'].fillna(-1)
df['dewey'] = df['dewey'].astype(int)

### Merge dewey dataset
We do a left join to get the nonfiction subjects

In [37]:
df = pd.merge(df, df_dewey, on='dewey', how='left')

#### Now that we have extracted the nonfiction subjects we can drop CallNumber and dewey

In [38]:
df.drop(['CallNumber', 'dewey'], axis=1, inplace=True)

In [40]:
df.head(4)

,BibNumber,CheckoutDateTime,CheckoutMonth,CheckoutDay,deweySubject
0,3178331,2016-07-30 11:24:00,7,5,NaN
1,2619159,2016-12-23 13:04:00,12,4,Arts & Recreation
2,2406194,2016-01-26 13:53:00,1,1,NaN
3,2655929,2016-06-03 16:56:00,6,4,Language


### Rename the column
BibNumber to BibNum so we can join on it easily

In [56]:
df.rename(columns={'BibNumber':'BibNum'}, inplace=True)

In [46]:
# df.to_csv('../../data/processed/checkouts.csv', index=False)

### Remove duplicates from Collection dataset
To do a proper join with the checkout records, the duplicate BibNums from all the different branches will have to be removed so that there is only one BibNum per item. Unfortunately, because the checkouts don't have a location column, there is no way to tell where the item was borrowed. Books have copies across many locations but they all have the same BibNum so there is no way to match to locations with BibNum alone.

In [48]:
df_inv.drop_duplicates(subset='BibNum',inplace=True)

In [54]:
print('Unique BibNum rows,cols',df_inv.shape)

Unique BibNum rows,cols (580934, 13)


#### Drop unecessary columns before join

In [59]:
df_inv.drop(['ItemLocation', 'ItemCount'], axis=1, inplace=True)

### Merge collection and checkout datasets

In [63]:
df = pd.merge(df, df_inv, on='BibNum', how='left')

In [64]:
df.isnull().sum()

BibNum                    0
CheckoutDateTime          0
CheckoutMonth             0
CheckoutDay               0
deweySubject        4081881
Title                188108
Author              2231196
Publisher            190149
Age                  187758
Description          187758
Format Group         187758
Format Subgroup      187758
Category Group      1311689
Child                187758
Teen                 187758
dtype: int64

In [71]:
print(len(df[df['Title'].isnull()==True]['BibNum'].value_counts().sort_values(ascending=False)))

29343


### Unable to match
It is quite a surprise to find that 187,758 checkout events for 29,343 items for 2016 did not find a match in the late 2017 collection. This means the library turns over quite a bit each year in order to accomodate new materials. It would certainly be preferable to find two datasets from the same timeframe to reduce this discrepancy although it is useful to see quite plainly how much change occurs within the collection.

The EDA will not be too affected by this issue. The timeseries aspect of the checkouts remains clean. There will be no choice but to ignore and drop the new nulls created after the general count analysis has been done. We can only work with the information we have and without any information about the items that didn't match they are not useful for further analysis. For the purposes of this project, when a model is chosen, it will have to be specific enough to avoid all the nulls. It is clear that the results of an inner join might be better when the time comes to implement a model.

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6404481 entries, 0 to 6404480
Data columns (total 15 columns):
BibNum              int64
CheckoutDateTime    datetime64[ns]
CheckoutMonth       int64
CheckoutDay         int64
deweySubject        object
Title               object
Author              object
Publisher           object
Age                 float64
Description         object
Format Group        object
Format Subgroup     object
Category Group      object
Child               float64
Teen                float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(8)
memory usage: 781.8+ MB


### Save merged file
This file will be used for EDA in the next notebook

In [73]:
# df.to_csv('../../data/processed/mergedlibrary.csv', index=False)